# Implement KNN from scratch in Python with Numpy

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

```
class MyKNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors=n_neighbors
        self.X_train=None
        self.Y_train=None
    def fit(self, X, Y):
        pass
    def predict(self, X):
        pass
```

In [73]:
fruits = pd.read_csv('fruit.csv')
X = fruits[['width', 'height', 'color_score']].values
Y = fruits['fruit_label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [75]:
[Y_train.min(), Y_train.max()]

[np.int64(1), np.int64(4)]

In [77]:
[Y_test.min(), Y_test.max()]

[np.int64(1), np.int64(4)]

In [79]:
Y_train=Y_train-1
Y_test=Y_test-1

In [81]:
[Y_train.min(), Y_train.max()]

[np.int64(0), np.int64(3)]

In [103]:
[Y_test.min(), Y_test.max()]

[np.int64(0), np.int64(3)]

In [105]:
num_classes=max(Y_train+1)
num_classes

np.int64(4)

### implement a function to find the K nearest neighbors
note: the code below is not the most efficient way to find the K nearest neighbors \
the purpose here is to enhance our programming sills in Numpy

In [108]:
X_train.shape

(47, 3)

In [37]:
X_test.shape

(12, 3)

In [39]:
#In the training set, find the nearest K neighbours of the sample X_test[0]

In [41]:
x0=X_test[0]
x0

array([9.6 , 9.2 , 0.74])

In [43]:
distance_array=np.zeros(X_train.shape[0])
for n in range(0, X_train.shape[0]):
    dist_n=np.sqrt(((x0-X_train[n])**2).sum())
    distance_array[n]=dist_n
distance_array[0:10]

array([3.02365673, 2.23662692, 2.37065392, 3.125124  , 2.46989878,
       6.44087727, 2.75907593, 3.25631694, 3.83276662, 3.16305232])

In [45]:
distance_array=np.sqrt(((x0.reshape(1,-1)-X_train)**2).sum(axis=1))
distance_array[0:10]

array([3.02365673, 2.23662692, 2.37065392, 3.125124  , 2.46989878,
       6.44087727, 2.75907593, 3.25631694, 3.83276662, 3.16305232])

In [47]:
sorted_index_array=np.argsort(distance_array)

In [49]:
distance_array[sorted_index_array[0]]

np.float64(0.40012498047485145)

In [51]:
distance_array[sorted_index_array[-1]]

np.float64(6.440877269440863)

In [53]:
K=5
K_nearest_neighbours_x0=sorted_index_array[0:K]
K_nearest_neighbours_x0

array([24, 33, 14, 17,  1])

In [55]:
Y_train[K_nearest_neighbours_x0]

array([2, 2, 2, 2, 2])

In [57]:
# majority voting among the K nearest neighbours of x0
counter=np.zeros(num_classes)
for k in range(0, K):
    class_id=Y_train[K_nearest_neighbours_x0][k]
    counter[class_id]+=1 # one vote for class_id
counter

array([0., 0., 5., 0.])

In [59]:
#predicted class label of the sample x0
y0_pred = np.argmax(counter)
y0_pred

np.int64(2)

In [61]:
def find_K_nearest_neighbours(X_train, X_test, K):
    # X_train.shape (N1, M)
    # X_test.shape (N2, M)
    K_nearest_neighbours=[]
    for n in range(0, X_test.shape[0]):
        x=X_test[n]
        distance_array=np.sqrt(((x.reshape(1,-1)-X_train)**2).sum(axis=1))
        sorted_index_array=np.argsort(distance_array)
        K_nearest_neighbours_x=sorted_index_array[0:K]
        K_nearest_neighbours.append(K_nearest_neighbours_x)
    K_nearest_neighbours=np.array(K_nearest_neighbours)
    return K_nearest_neighbours

In [63]:
K_nearest_neighbours=find_K_nearest_neighbours(X_train, X_test, K)

In [65]:
K_nearest_neighbours

array([[24, 33, 14, 17,  1],
       [29, 22, 23, 36,  0],
       [46,  4, 45, 32, 14],
       [40, 31,  9,  7,  3],
       [36,  0, 22,  9, 29],
       [10,  6, 38, 12,  3],
       [35, 25, 17, 18,  2],
       [ 4, 46, 45, 32, 14],
       [ 0, 36,  9, 10, 23],
       [39, 38,  3,  7, 12],
       [42, 34, 16,  5,  7],
       [ 9, 40, 31,  0,  3]])

In [67]:
K_nearest_neighbours.shape

(12, 5)

### implement a function to do majority voting

In [83]:
def majority_vote(K_nearest_neighbours, Y_train):
    Y_pred=np.zeros(K_nearest_neighbours.shape[0])
    K=K_nearest_neighbours.shape[1]
    num_classes=max(Y_train)-min(Y_train)+1
    for n in range(0, len(Y_pred)):
        counter=np.zeros(num_classes)
        for k in range(0, K):
            class_id=Y_train[K_nearest_neighbours[n]][k]
            counter[class_id]+=1 # one vote for class_id
        Y_pred[n]=np.argmax(counter)
    return Y_pred

In [85]:
Y_test_pred=majority_vote(K_nearest_neighbours, Y_train)

In [87]:
Y_test_pred

array([2., 2., 3., 2., 2., 0., 0., 3., 2., 0., 1., 2.])

### implement a function to caluclate classification accuracy

In [90]:
def cal_accuracy(Y_test, Y_test_pred):
    if Y_test.shape[0] != Y_test_pred.shape[0]:
        raise ValueError('wrong shapes')
    num_samples=Y_test.shape[0]
    correct=0
    for n in range(0, num_samples):
        if Y_test[n]==Y_test_pred[n]:
            correct+=1
    accuracy=correct/num_samples
    return accuracy

In [92]:
cal_accuracy(Y_test, Y_test_pred)

0.75

In [94]:
#one line of code to calculate accuracy
accuracy=(Y_test==Y_test_pred).sum()/Y_test.shape[0]
accuracy

np.float64(0.75)

### implement MyKNNClassifier

In [84]:
class MyKNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors=n_neighbors
        self.X_train=None
        self.Y_train=None
    def fit(self, X, Y):
        self.X_train=X
        self.Y_train=Y
        self.num_classes=max(Y)-min(Y)+1
    def predict(self, X):
        K_nearest_neighbours=find_K_nearest_neighbours(X_train=self.X_train, X_test=X, K=self.n_neighbors)
        Y_test_pred=majority_vote(K_nearest_neighbours, self.Y_train)
        return Y_test_pred

In [86]:
my_knn=MyKNNClassifier(n_neighbors=5)
my_knn.fit(X_train, Y_train)
Y_test_pred=my_knn.predict(X_test)
Y_test_pred

array([2., 2., 3., 2., 2., 0., 0., 3., 2., 0., 1., 2.])

In [88]:
#one line of code to calculate accuracy
accuracy=(Y_test==Y_test_pred).sum()/Y_test.shape[0]
accuracy

np.float64(0.75)